In [1]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
df=pd.read_csv('C:/Users/yy2895/Desktop/RawData.csv')
d = df.values
ntotal=len(df)


d = normalize(d, axis=0, norm='l2')


resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(7)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,8):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 14
n_encoder_h_2 = 10



#n_encoder_h_5 = 10


# 3 hidden layers for decoder
#n_decoder_h_1 = 10
n_decoder_h_1 = 10
n_decoder_h_2 = 14




# Parameters
learning_rate = 0.01

#batch_size = 7
display_step = 1

total_batch=7

training_epochs = 7000
#we store the variables here

def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [19, n_encoder_h_1], [n_encoder_h_1], phase_train)

        with tf.variable_scope("h_2"):
            h_2 = layer(h_1, [n_encoder_h_1, n_encoder_h_2], [n_encoder_h_2], phase_train)

       
        with tf.variable_scope("code"):
            output = layer(h_2, [n_encoder_h_2, n_code], [n_code], phase_train)

    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [n_code, n_decoder_h_1], [n_decoder_h_1], phase_train)

        with tf.variable_scope("h_2"):
            h_2 = layer(h_1, [n_decoder_h_1, n_decoder_h_2], [n_decoder_h_2], phase_train)


        with tf.variable_scope("output"):
            output = layer(h_2, [n_decoder_h_2, 19], [19], phase_train)

    return output
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')

    #if a python file, please use the 4 lines bellow and comment the "n_code = '2'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '8'
    
    #feel free to change with your own 
    #log_files_path = r'C:\Users\yy2895\Desktop\pys'
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 19]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive300=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, 'C:/Users/yy2895/Desktop/auto19-14-10-8/tmp/model.ckpt')
                        togive2=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,19)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive2.append(output_any_image)
                            
                        togive300=togive2
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)


we begin
Epoch: 0001 cost = 2.293776648
Validation Loss: 2.0525239
Epoch: 0002 cost = 2.270734753
Validation Loss: 2.489619
Epoch: 0003 cost = 2.248210430
Validation Loss: 2.2919824
Epoch: 0004 cost = 2.226138115
Validation Loss: 2.1569154
Epoch: 0005 cost = 2.205404282
Validation Loss: 2.040933
Epoch: 0006 cost = 2.185247592
Validation Loss: 1.9714667
Epoch: 0007 cost = 2.164665665
Validation Loss: 1.9393818
Epoch: 0008 cost = 2.143303905
Validation Loss: 1.981888
Epoch: 0009 cost = 2.120675666
Validation Loss: 2.156016
Epoch: 0010 cost = 2.097366435
Validation Loss: 2.2108324
Epoch: 0011 cost = 2.073696375
Validation Loss: 2.125171
Epoch: 0012 cost = 2.047821011
Validation Loss: 2.1118512
Epoch: 0013 cost = 2.022314548
Validation Loss: 1.9907997
Epoch: 0014 cost = 2.003979342
Validation Loss: 1.8899608
Epoch: 0015 cost = 1.990101389
Validation Loss: 1.4574552
Epoch: 0016 cost = 1.977715373
Validation Loss: 1.7107196
Epoch: 0017 cost = 1.966002328
Validation Loss: 1.6953595
Epoch: 001

Epoch: 0145 cost = 0.996654366
Validation Loss: 0.9449846
Epoch: 0146 cost = 0.991964485
Validation Loss: 0.95701885
Epoch: 0147 cost = 0.987310878
Validation Loss: 0.90276456
Epoch: 0148 cost = 0.982700092
Validation Loss: 0.9230325
Epoch: 0149 cost = 0.978144160
Validation Loss: 0.8852159
Epoch: 0150 cost = 0.973660358
Validation Loss: 0.8424064
Epoch: 0151 cost = 0.969260837
Validation Loss: 0.7834151
Epoch: 0152 cost = 0.964918367
Validation Loss: 1.1059681
Epoch: 0153 cost = 0.960564928
Validation Loss: 0.88536805
Epoch: 0154 cost = 0.956285255
Validation Loss: 1.151601
Epoch: 0155 cost = 0.952506840
Validation Loss: 1.1544863
Epoch: 0156 cost = 0.949838059
Validation Loss: 0.97931916
Epoch: 0157 cost = 0.947536971
Validation Loss: 1.1470574
Epoch: 0158 cost = 0.941986373
Validation Loss: 1.2024727
Epoch: 0159 cost = 0.935597394
Validation Loss: 1.3661317
Epoch: 0160 cost = 0.930855257
Validation Loss: 0.9091334
Epoch: 0161 cost = 0.926588731
Validation Loss: 1.0344824
Epoch: 0162

Validation Loss: 0.6579121
Epoch: 0288 cost = 0.562533694
Validation Loss: 0.7266982
Epoch: 0289 cost = 0.560530543
Validation Loss: 0.63777006
Epoch: 0290 cost = 0.558265865
Validation Loss: 0.5786736
Epoch: 0291 cost = 0.556362816
Validation Loss: 0.81262
Epoch: 0292 cost = 0.554424124
Validation Loss: 0.61784476
Epoch: 0293 cost = 0.552993604
Validation Loss: 0.650884
Epoch: 0294 cost = 0.550789722
Validation Loss: 0.5905485
Epoch: 0295 cost = 0.548807859
Validation Loss: 0.9695306
Epoch: 0296 cost = 0.546925528
Validation Loss: 0.88937867
Epoch: 0297 cost = 0.545103116
Validation Loss: 0.573752
Epoch: 0298 cost = 0.543258096
Validation Loss: 0.53450286
Epoch: 0299 cost = 0.541637216
Validation Loss: 0.56126815
Epoch: 0300 cost = 0.539757439
Validation Loss: 0.5340048
Epoch: 0301 cost = 0.538099085
Validation Loss: 0.4927017
Epoch: 0302 cost = 0.536353767
Validation Loss: 0.81563735
Epoch: 0303 cost = 0.534697890
Validation Loss: 0.6081642
Epoch: 0304 cost = 0.532954003
Validation L

Validation Loss: 0.38457623
Epoch: 0429 cost = 0.383139095
Validation Loss: 0.3995339
Epoch: 0430 cost = 0.382335114
Validation Loss: 0.41027895
Epoch: 0431 cost = 0.381548818
Validation Loss: 0.37843654
Epoch: 0432 cost = 0.380680970
Validation Loss: 0.39924648
Epoch: 0433 cost = 0.379811151
Validation Loss: 0.42816243
Epoch: 0434 cost = 0.379038070
Validation Loss: 0.4355433
Epoch: 0435 cost = 0.378248909
Validation Loss: 0.43662116
Epoch: 0436 cost = 0.377414188
Validation Loss: 0.44026583
Epoch: 0437 cost = 0.376622928
Validation Loss: 0.43846032
Epoch: 0438 cost = 0.375889833
Validation Loss: 0.43462035
Epoch: 0439 cost = 0.375106296
Validation Loss: 0.446842
Epoch: 0440 cost = 0.374260885
Validation Loss: 0.45287135
Epoch: 0441 cost = 0.373462796
Validation Loss: 0.3973757
Epoch: 0442 cost = 0.372750759
Validation Loss: 0.40482536
Epoch: 0443 cost = 0.371976963
Validation Loss: 0.42901736
Epoch: 0444 cost = 0.371174961
Validation Loss: 0.3955164
Epoch: 0445 cost = 0.370452868
Val

Validation Loss: 0.55735344
Epoch: 0571 cost = 0.301308415
Validation Loss: 0.42934534
Epoch: 0572 cost = 0.301001938
Validation Loss: 0.3375713
Epoch: 0573 cost = 0.300703557
Validation Loss: 0.3522821
Epoch: 0574 cost = 0.300299949
Validation Loss: 0.55153596
Epoch: 0575 cost = 0.299903086
Validation Loss: 0.633282
Epoch: 0576 cost = 0.299581770
Validation Loss: 0.4944961
Epoch: 0577 cost = 0.299259399
Validation Loss: 0.3741373
Epoch: 0578 cost = 0.298883268
Validation Loss: 0.359821
Epoch: 0579 cost = 0.298427575
Validation Loss: 0.37861818
Epoch: 0580 cost = 0.298031332
Validation Loss: 0.3458449
Epoch: 0581 cost = 0.297736315
Validation Loss: 0.3427234
Epoch: 0582 cost = 0.297467445
Validation Loss: 0.33611828
Epoch: 0583 cost = 0.297125818
Validation Loss: 0.32351765
Epoch: 0584 cost = 0.296725088
Validation Loss: 0.42871803
Epoch: 0585 cost = 0.296399602
Validation Loss: 0.3427349
Epoch: 0586 cost = 0.296121621
Validation Loss: 0.33985457
Epoch: 0587 cost = 0.295757539
Validati

...

Validation Loss: 0.03171191
Epoch: 6385 cost = 0.026378794
Validation Loss: 0.035148464
Epoch: 6386 cost = 0.026379593
Validation Loss: 0.03502944
Epoch: 6387 cost = 0.026377388
Validation Loss: 0.03204438
Epoch: 6388 cost = 0.026370576
Validation Loss: 0.030338004
Epoch: 6389 cost = 0.026361145
Validation Loss: 0.030119091
Epoch: 6390 cost = 0.026351680
Validation Loss: 0.03034203
Epoch: 6391 cost = 0.026345749
Validation Loss: 0.030191464
Epoch: 6392 cost = 0.026345051
Validation Loss: 0.028738467
Epoch: 6393 cost = 0.026347860
Validation Loss: 0.029259576
Epoch: 6394 cost = 0.026351285
Validation Loss: 0.029634172
Epoch: 6395 cost = 0.026353148
Validation Loss: 0.029851472
Epoch: 6396 cost = 0.026353123
Validation Loss: 0.028663442
Epoch: 6397 cost = 0.026351749
Validation Loss: 0.030338295
Epoch: 6398 cost = 0.026349636
Validation Loss: 0.030478252
Epoch: 6399 cost = 0.026347433
Validation Loss: 0.031101584
Epoch: 6400 cost = 0.026345962
Validation Loss: 0.03243416
Epoch: 6401 cost

Epoch: 6525 cost = 0.026348556
Validation Loss: 0.031916693
Epoch: 6526 cost = 0.026347630
Validation Loss: 0.029593
Epoch: 6527 cost = 0.026340232
Validation Loss: 0.030118141
Epoch: 6528 cost = 0.026330159
Validation Loss: 0.031594366
Epoch: 6529 cost = 0.026320052
Validation Loss: 0.030813484
Epoch: 6530 cost = 0.026315713
Validation Loss: 0.030292748
Epoch: 6531 cost = 0.026320657
Validation Loss: 0.03170192
Epoch: 6532 cost = 0.026331373
Validation Loss: 0.030011065
Epoch: 6533 cost = 0.026342311
Validation Loss: 0.030113429
Epoch: 6534 cost = 0.026350361
Validation Loss: 0.030294618
Epoch: 6535 cost = 0.026354247
Validation Loss: 0.029608008
Epoch: 6536 cost = 0.026354519
Validation Loss: 0.030822517
Epoch: 6537 cost = 0.026352370
Validation Loss: 0.029073767
Epoch: 6538 cost = 0.026349434
Validation Loss: 0.030039584
Epoch: 6539 cost = 0.026347916
Validation Loss: 0.03342081
Epoch: 6540 cost = 0.026348685
Validation Loss: 0.032761242
Epoch: 6541 cost = 0.026351310
Validation Los

Validation Loss: 0.03334503
Epoch: 6671 cost = 0.026295439
Validation Loss: 0.033659402
Epoch: 6672 cost = 0.026290699
Validation Loss: 0.031738386
Epoch: 6673 cost = 0.026284169
Validation Loss: 0.03197127
Epoch: 6674 cost = 0.026276945
Validation Loss: 0.03025123
Epoch: 6675 cost = 0.026269982
Validation Loss: 0.031183794
Epoch: 6676 cost = 0.026263576
Validation Loss: 0.034268033
Epoch: 6677 cost = 0.026258844
Validation Loss: 0.036168452
Epoch: 6678 cost = 0.026257415
Validation Loss: 0.03343485
Epoch: 6679 cost = 0.026260715
Validation Loss: 0.029478101
Epoch: 6680 cost = 0.026269195
Validation Loss: 0.029556066
Epoch: 6681 cost = 0.026281084
Validation Loss: 0.030454148
Epoch: 6682 cost = 0.026292658
Validation Loss: 0.0287515
Epoch: 6683 cost = 0.026299966
Validation Loss: 0.029994236
Epoch: 6684 cost = 0.026300749
Validation Loss: 0.030569665
Epoch: 6685 cost = 0.026294699
Validation Loss: 0.031456005
Epoch: 6686 cost = 0.026283188
Validation Loss: 0.03029088
Epoch: 6687 cost =

Epoch: 6811 cost = 0.026212015
Validation Loss: 0.03142392
Epoch: 6812 cost = 0.026209267
Validation Loss: 0.029778138
Epoch: 6813 cost = 0.026208520
Validation Loss: 0.029199887
Epoch: 6814 cost = 0.026209781
Validation Loss: 0.029742928
Epoch: 6815 cost = 0.026213849
Validation Loss: 0.03152784
Epoch: 6816 cost = 0.026221868
Validation Loss: 0.033119593
Epoch: 6817 cost = 0.026235619
Validation Loss: 0.033238642
Epoch: 6818 cost = 0.026257972
Validation Loss: 0.031778473
Epoch: 6819 cost = 0.026290852
Validation Loss: 0.036097337
Epoch: 6820 cost = 0.026322989
Validation Loss: 0.03496425
Epoch: 6821 cost = 0.026332420
Validation Loss: 0.032592196
Epoch: 6822 cost = 0.026321710
Validation Loss: 0.035228364
Epoch: 6823 cost = 0.026295158
Validation Loss: 0.03941702
Epoch: 6824 cost = 0.026267872
Validation Loss: 0.035700243
Epoch: 6825 cost = 0.026256457
Validation Loss: 0.031067142
Epoch: 6826 cost = 0.026255885
Validation Loss: 0.034267355
Epoch: 6827 cost = 0.026257301
Validation Lo

Validation Loss: 0.030864026
Epoch: 6954 cost = 0.026231540
Validation Loss: 0.030053396
Epoch: 6955 cost = 0.026216295
Validation Loss: 0.030239249
Epoch: 6956 cost = 0.026205325
Validation Loss: 0.030010339
Epoch: 6957 cost = 0.026201876
Validation Loss: 0.030754786
Epoch: 6958 cost = 0.026208769
Validation Loss: 0.030613707
Epoch: 6959 cost = 0.026221432
Validation Loss: 0.030673372
Epoch: 6960 cost = 0.026233746
Validation Loss: 0.03132501
Epoch: 6961 cost = 0.026241552
Validation Loss: 0.031335857
Epoch: 6962 cost = 0.026241731
Validation Loss: 0.034051232
Epoch: 6963 cost = 0.026234253
Validation Loss: 0.029634194
Epoch: 6964 cost = 0.026224254
Validation Loss: 0.029676888
Epoch: 6965 cost = 0.026217615
Validation Loss: 0.030188642
Epoch: 6966 cost = 0.026215440
Validation Loss: 0.030369112
Epoch: 6967 cost = 0.026215635
Validation Loss: 0.031215943
Epoch: 6968 cost = 0.026216689
Validation Loss: 0.031819146
Epoch: 6969 cost = 0.026218179
Validation Loss: 0.030151574
Epoch: 6970 

In [2]:
currentmin

0.028138287

In [3]:
import csv
with open('C:/Users/yy2895/Desktop/auto_result19-14-10-8.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive300)):
        wr.writerow(list(togive300[i][0]))

In [4]:
togive300

[array([[0.09445567, 0.77141815, 0.08361873, 0.02320707, 0.83944696,
         0.59230715, 0.85990864, 0.16303898]], dtype=float32),
 array([[0.1016905 , 0.7627086 , 0.09080416, 0.04004675, 0.84082675,
         0.5975297 , 0.8504625 , 0.17930992]], dtype=float32),
 array([[0.1455906 , 0.711112  , 0.13850509, 0.07426926, 0.8005044 ,
         0.5344819 , 0.8065007 , 0.21171796]], dtype=float32),
 array([[0.22682448, 0.65830594, 0.1845897 , 0.13127759, 0.73803174,
         0.58198965, 0.75410974, 0.26490292]], dtype=float32),
 array([[0.56999177, 0.46373597, 0.47252247, 0.5099546 , 0.47788936,
         0.4938351 , 0.5445726 , 0.47143126]], dtype=float32),
 array([[0.35478994, 0.61344355, 0.29869285, 0.3319199 , 0.6543291 ,
         0.47901607, 0.6665764 , 0.38153172]], dtype=float32),
 array([[0.6169701 , 0.3618024 , 0.5373668 , 0.62785023, 0.39152768,
         0.4106324 , 0.45696884, 0.5160686 ]], dtype=float32),
 array([[0.51136017, 0.44563115, 0.42271477, 0.676902  , 0.48017058,
       